##雲端硬碟設定指向

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls 'drive'

MyDrive  Shareddrives


In [3]:
import os
path = "/content/drive/My Drive/Colab Notebooks/BERT_Spacy3/"
os.chdir(path)

In [4]:
!ls

 1.Ner_converter_tea.ipynb
 1.RelationBinary_converter_tea.ipynb
 2.Train_Spacy_Bert_FineTuneNER_tea.ipynb
'2.Train_Spacy_Bert_Relation Extraction_tea.ipynb'
 3.Predict_tea.ipynb
 configs_ner
 cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
 data_ner
 data_relation
 outputs
 rel_component
 原文參考


In [5]:
!pip install typer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# 安裝 spacy
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#import sys
#print(sys.version)
import spacy
print(spacy.__version__)

3.3.1


In [8]:
#!pip uninstall spacy

In [9]:
import json

import typer
from pathlib import Path

from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
#from spacy.lang.en import English
from spacy.util import compile_infix_regex
import re
import spacy

In [10]:
# Create a blank Tokenizer with just the English vocab
nlp = spacy.blank("zh")

In [11]:
msg = Printer()
SYMM_LABELS = ["Binds"]
MAP_LABELS = {
    "別稱": "別稱",
    "英譯": "英譯",
    "類別": "類別",
    "生產於": "生產於",
    "分布於": "分布於",
    "是": "是",
    "適製": "適製",
    "屬於": "屬於",
    "採收於": "採收於",
    "命名於": "命名於",
    "母本": "母本",
    "父本": "父本",    
    "寄生於": "寄生於",
    "影響於": "影響於",
    "親本": "親本",
    "防治於": "防治於",
    "製造過程": "製造過程",
    "有": "有",
    "適合於": "適合於",
    "QA": "QA",
    "因果": "因果"
}

ann = "data_relation/relations_training.txt"
train_file='data_relation/relations_training.spacy'
dev_file='data_relation/relations_dev.spacy'
test_file='data_relation/relations_test.spacy'

In [12]:
def main(json_loc: Path, train_file: Path, dev_file: Path, test_file: Path , flag):
    """Creating the corpus from the Prodigy annotations."""
    print("Creating the corpus from the Prodigy annotations.")
    Doc.set_extension("rel", default={},force=True)
    vocab = Vocab()

    docs = {"train": [], "dev": [], "test": [], "total": []}
    ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
    count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
    count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}

    with open(json_loc, encoding="utf8") as jsonfile:
        print("load json start")
        file = json.load(jsonfile)
        for example in file:
            span_starts = set()
            neg = 0
            pos = 0
            # Parse the tokens
            tokens=nlp(example["document"])    

            spaces=[]
            spaces = [True if tok.whitespace_ else False for tok in tokens]
            words = [t.text for t in tokens]
            doc = Doc(nlp.vocab, words=words, spaces=spaces)


            # Parse the GGP entities
            spans = example["tokens"]
            entities = []
            span_end_to_start = {}
            for span in spans:
                entity = doc.char_span(
                     span["start"], span["end"], label=span["entityLabel"]
                 )
                # 多加入NoneType判斷
                if entity is None:
                  print("class is NoneType")
                else:                
                  span_end_to_start[span["token_start"]] = span["token_start"]
                  #print(span_end_to_start)
                  entities.append(entity)
                  span_starts.add(span["token_start"])               

            doc.ents = entities

            # Parse the relations
            rels = {}
            for x1 in span_starts:
                for x2 in span_starts:
                    rels[(x1, x2)] = {}
                    #print(rels)
            relations = example["relations"]
            #print(relations)
            #print(len(relations))
            for relation in relations:
                try:
                  # the 'head' and 'child' annotations refer to the end token in the span
                  # but we want the first token
                  start = span_end_to_start[relation["head"]]
                  end = span_end_to_start[relation["child"]]
                  label = relation["relationLabel"]
                  #print(rels[(start, end)])
                  #print(label)
                  #label = MAP_LABELS[label]
                  if label not in rels[(start, end)]:
                      rels[(start, end)][label] = 1.0
                      pos += 1
                      #print(pos)
                      #print(rels[(start, end)])
                except:
                  print("發生錯誤:key error")                     

            # The annotation is complete, so fill in zero's where the data is missing
            for x1 in span_starts:
                for x2 in span_starts:
                    for label in MAP_LABELS.values():
                        if label not in rels[(x1, x2)]:
                            neg += 1
                            rels[(x1, x2)][label] = 0.0

                            #print(rels[(x1, x2)])
            doc._.rel = rels
            #print(doc._.rel)

            # only keeping documents with at least 1 positive case
            if pos > 0:
                    docs["total"].append(doc)
                    count_pos["total"] += pos
                    count_all["total"] += pos + neg

                    
                    
    print("=====================================")
    print(len(docs["total"]))
    #print(docs["total"])
    docbin = DocBin(docs=docs["total"], store_user_data=True)
    if flag == "train":
        docbin.to_disk(train_file)
    elif flag == "dev":
        docbin.to_disk(dev_file)
    elif flag == "test":
        docbin.to_disk(test_file)
    else:
        print("flag error")
    msg.info(
        f"{len(docs['total'])} training sentences"
    )

In [13]:
main(ann, train_file, dev_file, test_file , "train" )

Creating the corpus from the Prodigy annotations.
load json start
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
class is NoneType
發生錯誤:key error
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
class is NoneType
class is NoneType
發生錯誤:key error
發生錯誤:key error
111
ℹ 111 training sentences


In [14]:
ann = "data_relation/relations_dev.txt"
main(ann, train_file, dev_file, test_file , "dev" )

Creating the corpus from the Prodigy annotations.
load json start
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
class is NoneType
發生錯誤:key error
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
class is NoneType
發生錯誤:key error
class is NoneType
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
發生錯誤:key error
class is NoneType
class is NoneType
發生錯誤:key error
發生錯誤:key error
111
ℹ 111 training sentences


In [15]:
ann = "data_relation/relations_test.txt"
main(ann, train_file, dev_file, test_file , "test" )

Creating the corpus from the Prodigy annotations.


FileNotFoundError: ignored